In [1]:
import os # 운영체제와 상호작용 할 수 있는 방법을 제공하는 모델
from dotenv import load_dotenv # env파일 안에있는 API키를 가져오기 위한 라이브러리

In [2]:
load_dotenv() # API키 가져오기

True

In [4]:
# langchain을 통해 openai의 gpt모델 사용
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")


In [8]:
! pip install pypdf

In [5]:
# pdf파일을 불러오기 위한 PyPDFLoader를 import
from langchain.document_loaders import PyPDFLoader

In [6]:
# pdf파일 로드
loader = PyPDFLoader("초거대 언어모델 연구 동향 (1).pdf")

# 페이지별 문서 로드
docs = loader.load()

In [ ]:
# 문서 청크로 나누기

# 방법1

# 청킹을 위한 라이브러리 imoprt
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n", # /n/n 기준으로 분할
    chunk_size=100,   # 청크 크기는 100자
    chunk_overlap=10, # 청크 간 중복된 문자는 10개
    length_function=len,  # len 함수로 텍스트의 길이 계산(문자 수 반환)
    is_separator_regex=False,  # 구분자로 정규식을 사용하지 않는다
)

splits = text_splitter.split_documents(docs)

In [10]:
print(splits[2])

page_content='10 특집원고  초거대 언어모델 연구 동향
치하는 정보들을 기억하지 못하는 장기 의존성 문제
가 존재한다. 이러한 문제를 극복하기 위해, LSTM 
(Long Short-Term Memory) [14] 과 GRU (Gated 
Recurrent Unit) [15] 가 등장했다. 하지만, 이들은 모두 
텍스트에 존재하는 단방향 문맥 정보만 활용한다는 
한계를 지닌다.
양방향 문맥 정보를 활용하기 위해, ELMo [16] 는 
주어진 텍스트에 존재하는 순방향 문맥 정보와 역방
향 문맥 정보를 함께 활용하는 양방향 학습을 제안했
다. 이를 위해, ELMo 는 순방 향 LSTM과 역방향 
LSTM를 동시에 활용한다. 하지만, 이는 LSTM을 기
반으로 하기 때문에, LSTM이 지니는 다음과 같은 한
계를 그대로 가진다: 1) 하나의 벡터에 텍스트의 모든 
정보를 담기 때문에 정보 손실이 발생하고, 2) 입력 
텍스트의 길이가 길어지면 기울기 소실 (gradient 
vanishing)이 발생한다.
이러한 한계를 해결하기 위해 나온 것이 바로 
Attention Mechanism [2] 과 이를 활용한 Transformer 
Architecture [3] 이다. Attention Mechanism 은 하나의 
벡터에 텍스트의 모든 정보를 담는 RNN, LSTM, 
GRU와 다르게, 텍스트 내 단어들의 벡터들을 필요에 
따라 적절히 활용하는 메커니즘이다. 현재 언어모델
의 근간이 되는 Transformer가 바로 이러한 Atten- 
tion Mechanism을 기반으로 한다. Transformer는 크게 
인코더와 디코더로 구성되는데, 인코더는 주어진 텍
스트를 이해하는 역할을 하고 디코더는 이해한 텍스
트를 기반으로 언어를 생성해내는 역할을 수행한다. 
이러한 Transformer의 인코더를 기반으로 발전한 대표
적인 모델이 Google2)의 BERT (Bidirectional Encoder 
Representations from Transf

## CharacterTextSplitter
주어진 텍스트를 기준으로 정의된 구분자를 사용하여 텍스트를 나눈다. 특정 문자를 기준으로 분할해서 문장이나 문단 단위로 텍스트를 나누는데 효과적(간단한 작업, 빠르고 직관적인 경우 유리)
>
- separator
분할된 각 청크를 구분할 때 기준이 되는 문자열
>
- chunk_size
각 청크의 최대 길이
>
- chunk_overlap
인접한 청크 사이에 중복으로 포함될 문자의 수
>
- length_function
청크의 길이를 계산하는 함수
>
- is_separator_regex
매개변수를 False로 설정하여 separator를 정규식이 아닌 일반 문자열로 처리
>
#### 각 청크의 크기가 chunk_size를 초과하지 않으며 인접한 청크 사이에는 chunk_overlap만큼의 문자가 중복되어 있게 함으로써, 텍스트의 의미적 연속성을 유지하면서 큰 데이터를 더 작은 단위로 분할할 수 있다
>
- text_splitter 
>
1. split_documents
문서(document) 파일을 load한 후 다시 작은 단위조각의 문서(document) list로 반환
>
2. split_text
문자 스트링(str)을 list로 반환
>
3. create_documents
문자열list를 문서(document)list로 반환



In [ ]:
# 방법2

# 청킹을 위한 라이브러리 imoprt
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,             # 청크 크기는 100자
    chunk_overlap=10,           # 청크 간의 중복되는 문자는 10개
    length_function=len,        # len함수로 텍스트의 길이 계산
    is_separator_regex=False,   # 구분자로 정규식을 사용하지 않는다
)

splits = recursive_text_splitter.split_documents(docs)

In [13]:
print(splits[2])

page_content='던 다양한 자연언어처리 하위 분야들이 하나의 모델
로 처리되고 있으며, 태스크 수렴 현상 (Converge)이 
발생하고 있다. 즉 하나의 LLM으로 번역, 요약, 질의' metadata={'source': '초거대 언어모델 연구 동향 (1).pdf', 'page': 0}


## RecursiveCharacterTextSplitter
여러 레벨의 구분자를 정의하여 점진적으로 텍스트를 나누는 방식(재귀적으로 분할). e.g. 문단으로 나눈 뒤 문단이 길면 문장 단위로, 문장이 여전히 길면 단어 단위로 나눔(긴 텍스트, 문맥 유지, 복잡한 텍스트 구조에 유리)
>
- chunk_size
청크 크기
>
- chunk_overlap
청크 간의 중복되는 문자 수
>
- length_function
문자열 길이를 계산하는 함수 지정
>
- is_separator_regex
구분자로 정규식을 사용할지 여부